In [6]:
suppressPackageStartupMessages({
    library(optparse)
    library(SpatialExperiment)
    library(Seurat)
    library(maple)
})

In [18]:
test_dir="/home/jovyan/scratch/SpaceHack2/userfolders/gmoranzoni/maple" # output_dir, can be modified

test_data="/home/jovyan/scratch/SpaceHack2/data/LIBD_DLPFC/Br5292_151507"
test_data_processed="/home/jovyan/scratch/SpaceHack2/method_results/LIBD_DLPFC/Br5292_151507"

dimred_file <- file.path(test_data_processed, "log1p/hvg/pca_20/dimensionality_reduction.tsv")
feature_file <- file.path(test_data_processed, "log1p/hvg/features.tsv")
observation_file <- file.path(test_data, "observations.tsv")
neighbor_file <- file.path(test_data_processed, "/delaunay/neighbors.mtx")
coord_file <- file.path(test_data, "coordinates.tsv")
matrix_file <- file.path(test_data, "counts.mtx")
n_clusters <- 7
technology <- "Visium"
seed <- 42 

In [19]:
# You can get SpatialExperiment directly
get_SpatialExperiment <- function(
    feature_file,
    observation_file,
    coord_file,
    matrix_file = NA,
    reducedDim_file = NA,
    assay_name = "counts",
    reducedDim_name = "PCs") {
  rowData <- read.delim(feature_file, stringsAsFactors = FALSE, row.names = 1)
  colData <- read.delim(observation_file, stringsAsFactors = FALSE, row.names = 1)

  coordinates <- read.delim(coord_file, sep = "\t", row.names = 1)
  coordinates <- as.matrix(coordinates[rownames(colData), ])
  coordinates[,c(1:2)] <- as.numeric(coordinates[,c(1:2)])

    
    if (!is.na(reducedDim_file)) {
    dimRed <- read.delim(reducedDim_file, stringsAsFactors = FALSE, row.names = 1)
    #reducedDim(spe, reducedDim_name) <- as.matrix(dimRed[colnames(spe), ])
  }
  spe <- SpatialExperiment::SpatialExperiment(
    rowData = rowData, colData = colData, spatialCoords = coordinates,
      reducedDims = list(pca = as.matrix(dimRed[rownames(colData), ]))
  )

  if (!is.na(matrix_file)) {
    assay(spe, "counts", withDimnames = FALSE) <- as(Matrix::t(Matrix::readMM(matrix_file)), "CsparseMatrix")
    assay(spe, "logcounts", withDimnames = FALSE) <- as(Matrix::t(Matrix::readMM(matrix_file)), "CsparseMatrix")
  }

  # Filter features and samples
  if ("selected" %in% colnames(rowData(spe))) {
    spe <- spe[as.logical(rowData(spe)$selected), ]
  }
  if ("selected" %in% colnames(colData(spe))) {
    spe <- spe[, as.logical(colData(spe)$selected)]
  }

  
  return(spe)
}

In [22]:
# You can use the data as SpatialExperiment
spe <- get_SpatialExperiment(feature_file = feature_file,observation_file = observation_file,
                                    coord_file = coord_file, reducedDim_file = dimred_file, 
                             matrix_file = matrix_file)

## Your code goes here
# TODO
seurat_obj <- as.Seurat(spe)

#coordinates <- read.delim(coord_file, sep = "\t", row.names = 1)
#coordinates <- as.matrix(coordinates[rownames(colData(spe)), ])
#coordinates[,c(1:2)] <- as.numeric(coordinates[,c(1:2)])
#rownames(coordinates) <- c("row", "col")

    # get spatial coordinates
#xy <- c("image_row", "image_col")
#if (!all(xy %in% names(colData)))
#  xy <- c("x", "y")
#xy <- colData[xy]
#colnames(xy) <- c("row", "col")

# Insert image coordinates
seurat_obj@images$image =  new(
  Class = 'SlideSeq',
  assay = "Spatial",
  key = "coordinates_",
  coordinates = as.data.frame(colData(spe))
)


# Run spruce
maple_results <- fit_maple(
  seurat_obj, 
  K = n_clusters, 
  emb = "PCs", 
  covars = "sample_id" #, n_dim = d 
  )

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from pca to pca_”


[1] "Dispatching to appropriate model fit function"
[1] "Started MCMC of 2000"
  |======================================================================| 100%
[1] "Finished MCMC after 2485.409 seconds"


In [44]:
embedding_df <- as.data.frame(maple_results$Y)

In [40]:
# save data
label_df <- data.frame("label" = maple_results$z, row.names=colnames(seurat_obj))
# The data.frames with observations may contain a column "selected" which you need to use to
# subset and also use to subset coordinates, neighbors, (transformed) count matrix
# label_df = ...  # data.frame with row.names (cell-id/barcode) and 1 column (label)
# embedding_df = NULL  # optional, data.frame with row.names (cell-id/barcode) and n columns

In [ ]:
## Write output
dir.create(out_dir, showWarnings = FALSE, recursive = TRUE)

colnames(label_df) <- c("label")
write.table(label_df, file = label_file, sep = "\t", col.names = NA, quote = FALSE)
if (exists("embedding_df")) {
  write.table(embedding_df, file = embedding_file, sep = "\t", col.names = NA, quote = FALSE)
}